In [ ]:
from graph_tool.all import *
import graph_tool.all as gt
import scipy as sp
import numpy as np
from tqdm import tqdm
import os

In [ ]:
Node = 5000

m = 2

times = 10

beta_list = [1,-1]

name_list = ["SF"]

In [ ]:
# 第二固有値と固有ベクトルを算出する関数
def no_normalized_laplacian(g):
    L = gt.laplacian(g)
    L_sparse = sp.sparse.csr_matrix(L)
    eigenvalues, eigenvectors = sp.sparse.linalg.eigsh(L_sparse, k=2, which='SM')
    return eigenvalues[1], eigenvectors[:, 1]

def top_k_max_abs_fiedler(fiedler_vector, g, k):
    """
    フィードラーベクトルの差分が大きい順に最大 k 個のノードペアを取得
    """
    # フィードラーベクトルを1次元配列として取得
    fiedler_vector = np.array(fiedler_vector)
    
    # 差分行列の計算（|fiedler[i] - fiedler[j]|）
    diff_matrix = np.abs(fiedler_vector[:, None] - fiedler_vector[None, :])
    
    # 隣接行列を取得
    adjacency = gt.adjacency(g).toarray()
    
    # エッジがないノードペアのみを対象とするフィルタ
    mask = (adjacency == 0)  # 隣接していないノードペアが True
    
    # 差分行列にマスクを適用
    diff_matrix[~mask] = -np.inf  # エッジが存在する部分は無効化 (-∞)
    
    # 差分行列を1次元に展開
    flattened_diff = diff_matrix.flatten()
    
    # 大きい順に k 個のインデックスを取得
    top_k_indices = np.argpartition(flattened_diff, -k)[-k:]
    top_k_indices = top_k_indices[np.argsort(flattened_diff[top_k_indices])[::-1]]
    
    # インデックスを2次元に戻し、ノードペアに変換
    node_pairs = [tuple(map(int, np.unravel_index(idx, diff_matrix.shape))) for idx in top_k_indices]
    
    return node_pairs


def top_k_min_abs_fiedler(fiedler_vector, g, k):
    """
    フィードラーベクトルの差分が小さい順に最小 k 個のノードペアを取得
    """
    # フィードラーベクトルを1次元配列として取得
    fiedler_vector = np.array(fiedler_vector)
    
    # グラフのエッジリストを取得（削除されたエッジは除外）
    edges = [(e.source(), e.target()) for e in g.edges()]
    
    # エッジリストからノードペアのインデックスを抽出
    node_pairs = np.array(edges, dtype=int)  # データ型を明示的に整数型に変換
    
    # 各エッジに対応するノードペアの差分を計算
    diffs = np.abs(fiedler_vector[node_pairs[:, 0]] - fiedler_vector[node_pairs[:, 1]])
    
    # 小さい順に k 個のインデックスを取得
    top_k_indices = np.argpartition(diffs, k)[:k]
    top_k_indices = top_k_indices[np.argsort(diffs[top_k_indices])]
    
    # インデックスからノードペアに変換
    top_k_node_pairs = [tuple(node_pairs[idx]) for idx in top_k_indices]
    
    return top_k_node_pairs

# 最大連結成分を求める
def max_component(g):
    l = gt.label_largest_component(g)
    u = gt.GraphView(g, vfilt=l)
    sum_u = u.num_vertices()
    return sum_u

In [ ]:
#count_list = [1, 2, 5, 20, 50, 100, 200, 500, 1000, 2500, 5000]
#recalculate_list = [5000, 2500, 1000, 250, 100, 50, 25, 10, 5, 2, 1]

count_list = [2]

recalculate_list = [2500]

In [ ]:
SF_μ_List = []
ER_μ_List = []
for count, recalculate in zip(count_list, recalculate_list):
    
    for net in name_list:
        
        if "SF" in net:
            SF_μ_list = []
        else:
            ER_μ_list = []
            
        for i in range(times):
            μ_list = []
            
            g = load_graph(f"~/o_t_hayashilab/Network_data/graph-tool/pure_pk/N={Node}/{net}/{i + 7 * times}.gt.gz")
            
            # 初期グラフを保存する前にコピーする
            G = g.copy()
            
            μ, fiedler_vector = no_normalized_laplacian(G)
            
            μ_list.append(μ)
            
            for _ in tqdm(range(recalculate)):
                non_link_result = top_k_max_abs_fiedler(fiedler_vector, G, k=count)
                
                for j, k in non_link_result:
                    G.add_edge(G.vertex(j), G.vertex(k))
                    
                μ, fiedler_vector = no_normalized_laplacian(G)
                    
                link_result = top_k_min_abs_fiedler(fiedler_vector, G, k=9990)
                # link_result の下位ノードペアのリンクを削除
                attempt = 0
                index = 0
                
                while attempt < count:
                    l, m = link_result[index]  # 現在のインデックスのノードペアを取得
                    edge_to_remove = G.edge(G.vertex(l), G.vertex(m))
                    G.remove_edge(edge_to_remove)
            
                    # 最大連結成分のノード数をチェック
                    if max_component(G) == Node:
                        link_result.pop(index)  # リストから要素を削除
                        attempt += 1
                        index = 0  # インデックスをリセットして次の削除操作を開始
                    else:
                        # リンクを復元
                        G.add_edge(G.vertex(l), G.vertex(m))
                        index += 1  # インデックスを進める
                        #print(f"{attempt}回目")
                        #print("最大連結成分崩れたためやり直し")
            
                    # インデックスがリンクリストの範囲を超えないように調整
                    if index >= len(link_result):
                        index = 0
                        
                μ, fiedler_vector = no_normalized_laplacian(G)
                
                μ_list.append(μ)
                        
            if "SF" in net:
                directory = (f"~/o_t_hayashilab/Network_data/graph-tool/rewire_laplacian/N={Node}/SF/c={count}_re={recalculate}/")
                os.makedirs(os.path.expanduser(directory), exist_ok=True)
                
                filename = directory + f"{i + 7 * times}.gt.gz"
                # ネットワークを保存
                G.save(filename)
                
                # 保存ディレクトリのパスを指定
                sf_save_dir = f"~/o_t_hayashilab/Network_information/rewire_laplacian/μ_list/N={Node}/SF/c={count}_re={recalculate}"

                # ディレクトリが存在しない場合は作成
                os.makedirs(os.path.expanduser(sf_save_dir), exist_ok=True)

                # 保存ファイル名
                sf_save_path = os.path.join(sf_save_dir, f"{i + 7 * times}.npy")

                # 1次元配列を2次元配列に変換（1行に並べる形にする）
                SF_μ_2D = np.reshape(μ_list, (1, -1))

                # SF_μ_ListとER_μ_Listをnumpy配列に変換して保存
                np.save(os.path.expanduser(sf_save_path), SF_μ_2D)

                SF_μ_list.append(μ_list)
            else:
                directory = (f"~/o_t_hayashilab/Network_data/graph-tool/rewire_laplacian/N={Node}/ER/c={count}_re={recalculate}/")
                os.makedirs(os.path.expanduser(directory), exist_ok=True)
                
                filename = directory + f"{i + 7 * times}.gt.gz"
                # ネットワークを保存
                G.save(filename)
                
                # 保存ディレクトリのパスを指定
                er_save_dir = f"~/o_t_hayashilab/Network_information/rewire_laplacian/μ_list/N={Node}/ER/c={count}_re={recalculate}"

                # ディレクトリが存在しない場合は作成
                os.makedirs(os.path.expanduser(er_save_dir), exist_ok=True)

                # 保存ファイル名
                er_save_path = os.path.join(er_save_dir, f"{i + 7 * times}.npy")

                # 1次元配列を2次元配列に変換（1行に並べる形にする）
                ER_μ_2D = np.reshape(μ_list, (1, -1))

                # SF_μ_ListとER_μ_Listをnumpy配列に変換して保存
                np.save(os.path.expanduser(er_save_path), ER_μ_2D)

                ER_μ_list.append(μ_list)
                
        if "SF" in net:
            SF = np.mean(SF_μ_list, axis=0)
            SF_μ_List.append(SF)
        else:
            ER = np.mean(ER_μ_list, axis=0)
            ER_μ_List.append(ER)

In [ ]:
# 保存ディレクトリのパスを指定
sf_save_dir = f"~/o_t_hayashilab/Network_information/rewire_laplacian/iteration_μ/N={Node}/SF"
#er_save_dir = f"~/o_t_hayashilab/Network_information/rewire_laplacian/iteration_μ/N={Node}/ER"

# ディレクトリが存在しない場合は作成
os.makedirs(os.path.expanduser(sf_save_dir), exist_ok=True)
#os.makedirs(os.path.expanduser(er_save_dir), exist_ok=True)

# 保存ファイル名
sf_save_path = os.path.join(sf_save_dir, f"c=5_re=1000_82-90.npy")
#er_save_path = os.path.join(er_save_dir, f"c=5_re=1000.npy")

# 1次元配列を2次元配列に変換（1行に並べる形にする）
SF_μ_2D = np.reshape(SF_μ_List[0], (1, -1))
#ER_μ_2D = np.reshape(ER_μ_List[0], (1, -1))

# SF_μ_ListとER_μ_Listをnumpy配列に変換して保存
np.save(os.path.expanduser(sf_save_path), SF_μ_2D)
#np.save(os.path.expanduser(er_save_path), ER_μ_2D)